In [1]:
import torch
import torch.nn as nn
from torchvision import models

# workaround for ssl errors:
from torchvision.models.densenet import model_urls
model_urls['densenet161'] = model_urls['densenet161'].replace('https://', 'http://')
densenet161model = models.densenet161(pretrained=True)

Downloading: "http://download.pytorch.org/models/densenet161-8d451a50.pth" to /Users/Christoph/.torch/models/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:07<00:00, 14967701.87it/s]


In [2]:
densenet161model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d (3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU(inplace)
        (conv.1): Conv2d (96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU(inplace)
        (conv.2): Conv2d (192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm.1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU(inplace)
        (conv.1): Conv2d (144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
   

In [3]:
# final batchnorm layer

num_features = densenet161model.features.norm5.num_features
#densenet161model.features.norm5 = nn.BatchNorm2d(num_features)

In [5]:
# Classifier

num_classes = 24
in_features = densenet161model.classifier.in_features
#densenet161model.classifier = nn.Linear(in_features, num_classes)

In [6]:
# remove final layers

features = nn.Sequential(*list(densenet161model.features.children())[:-2]) # remove dense_block 4 and following: [:-2]

In [7]:
features

Sequential (
  (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU (inplace)
  (3): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (4): _DenseBlock (
    (denselayer1): _DenseLayer (
      (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU (inplace)
      (conv.1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
      (relu.2): ReLU (inplace)
      (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer (
      (norm.1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU (inplace)
      (conv.1): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
      (relu.2): Re

In [8]:
features[9] # transition int 4th dense block

_Transition (
  (norm): BatchNorm2d(2112, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (conv): Conv2d(2112, 1056, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (pool): AvgPool2d (size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
)

In [9]:
# split denseblock 4 into part to be trained and freezed

# number of dense net block 4 layers to be removed: n (negative!)
# number of dense net block 4 layers to be kept: 0 ... n
# n element [0,...,23]
n = 10
    
feature4 = nn.Sequential(*list(densenet161model.features.denseblock4.children())[:n]) 
trainyou_wewill = nn.Sequential(*list(densenet161model.features.denseblock4.children())[n:]) 
feature4

Sequential (
  (0): _DenseLayer (
    (norm.1): BatchNorm2d(1056, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(1056, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (relu.2): ReLU (inplace)
    (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (1): _DenseLayer (
    (norm.1): BatchNorm2d(1104, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(1104, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (relu.2): ReLU (inplace)
    (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (2): _DenseLayer (
    (norm.1): BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(1152, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [10]:
trainyou_wewill

Sequential (
  (0): _DenseLayer (
    (norm.1): BatchNorm2d(1536, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(1536, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (relu.2): ReLU (inplace)
    (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (1): _DenseLayer (
    (norm.1): BatchNorm2d(1584, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(1584, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (relu.2): ReLU (inplace)
    (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (2): _DenseLayer (
    (norm.1): BatchNorm2d(1632, eps=1e-05, momentum=0.1, affine=True)
    (relu.1): ReLU (inplace)
    (conv.1): Conv2d(1632, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [11]:
# define feature extractor
features = nn.Sequential(features,feature4)

In [12]:
features

Sequential (
  (0): Sequential (
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (4): _DenseBlock (
      (denselayer1): _DenseLayer (
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU (inplace)
        (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer (
        (norm.1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(192, 

In [13]:
# define layers to be trained including BatchNorm
trainyou_wewill = nn.Sequential(trainyou_wewill, nn.BatchNorm2d(num_features))
# define classifier
classifier = nn.Linear(in_features, num_classes)

In [14]:
trainyou_wewill

Sequential (
  (0): Sequential (
    (0): _DenseLayer (
      (norm.1): BatchNorm2d(1536, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU (inplace)
      (conv.1): Conv2d(1536, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
      (relu.2): ReLU (inplace)
      (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): _DenseLayer (
      (norm.1): BatchNorm2d(1584, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU (inplace)
      (conv.1): Conv2d(1584, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm.2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
      (relu.2): ReLU (inplace)
      (conv.2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): _DenseLayer (
      (norm.1): BatchNorm2d(1632, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU (inplace)
      (conv.1): Conv2

In [15]:
#Convolution

#resetlayer = model.features.conv0

# extract parameters
#in_channels = resetlayer.in_channels
#out_channels = resetlayer.out_channels
#kernel_size = resetlayer.kernel_size
#stride = resetlayer.stride
#padding = resetlayer.padding
#bias = resetlayer.bias

# create new layer
#nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)

In [16]:
#Normalization

#resetlayer = model.features.norm0

# extract parameters
#num_features = resetlayer.num_features
#eps = resetlayer.eps
#momentum = resetlayer.eps
#affine = resetlayer.affine

# create new layer
#nn.BatchNorm2d(num_features, eps, momentum, affine=True)

In [17]:
#denselayer = model.features.denseblock1.denselayer1
#denselayer

In [18]:
#denselayer._modules["relu.1"]